# ETL (Extract, Load, Transform)

### 1. Importar Librerías

In [51]:
import pandas as pd
import numpy as np 
import seaborn as sn
from functions import cargar_excel
from functions import detectar_sd
from functions import limpiar_data
from sqlalchemy import create_engine

import datetime

import warnings
# Suppress warnings for specific categories (e.g., DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

Procedemos a leer los datasets y a revisar la forma como se visualizan usando la librería pandas, de manera que podamos decidir las transformaciones necesarias. 

In [4]:
df_homicidios_hechos = pd.read_excel("C:\PI2_DA\DataSets\homicidios.xlsx")
df_homicidios_victimas = pd.read_excel("C:\PI2_DA\DataSets\homicidios.xlsx", sheet_name="VICTIMAS")
df_lesiones_hechos = pd.read_excel("C:\PI2_DA\DataSets\lesiones.xlsx")
df_lesiones_victimas = pd.read_excel("C:\PI2_DA\DataSets\lesiones.xlsx", sheet_name="VICTIMAS")

In [5]:
df_lesiones_victimas.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,43466,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,43466,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,43466,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,43466,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,43466,sd,Mujer,45,SD


In [6]:
df_homicidios_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,42370,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,42370
1,2016-0002,42371,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,42371
2,2016-0003,42372,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,42372
3,2016-0004,42379,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,42390,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,42401


### CARGA DE DATOS
Usando una función creada en otro notebook y que se encuentra en el archivo "functions.py", realizaré el proceso de carga de una forma diferente, para optimizar el tratamiento de los datos. 

In [2]:
# Invoca la función 'cargar_excel'

data_homicidios = cargar_excel('C:\PI2_DA\DataSets\homicidios.xlsx', ['HECHOS', 'VICTIMAS'])
data_lesiones = cargar_excel('C:\PI2_DA\DataSets\lesiones.xlsx', ['HECHOS', 'VICTIMAS'])

# EXPLORACIÓN Y LIMPIEZA DE DATOS

### 3.1. Dataset: Homicidios / Hechos

In [52]:
# Accederé al dataframe "data_homicidios" en la hoja hechos y los cargaré en una variable exclusiva

homi_hechos = data_homicidios['HECHOS']

In [53]:
homi_hechos.info()
homi_hechos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Se verifica el diccionario de datos y se establece que los valores SD significa "sin datos". Para ello vamos a resolver todos los valores SD como NaN.

In [54]:
# Invoca la función 'detectar_sd'

rest_homi_hechos = detectar_sd(homi_hechos)
rest_homi_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,HORA,1,0.143678
7,HH,1,0.143678
8,LUGAR_DEL_HECHO,1,0.143678
19,VICTIMA,9,1.293103
20,ACUSADO,23,3.304598


In [55]:
# Reemplazar 'SD' por NaN en todo el DataFrame

homi_hechos.replace(['SD','sd'], pd.NA, inplace=True)

#### Función de limpieza de datos
Esta función "limpiar_datos", que permite realizar una amplia variedad de tareas de limpieza de datos en un DataFrame de pandas, proporcionando flexibilidad para manejar duplicados, valores nulos, formatos de datos, y más, mediante el uso de parámetros opcionales.

In [56]:
#Preparamos los diccionarios que serán intervenidos

# Columnas para pasar a minúcula

columns_to_lower = ['TIPO_DE_CALLE', 'VICTIMA', 'ACUSADO']

# Columnas para eliminar

columns_to_drop = ['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO','Calle','Altura',
                   'Cruce','Dirección Normalizada','XY (CABA)','PARTICIPANTES']

# Renombrar columnas

rename_dict = {'ID': 'id_siniestro', 'N_VICTIMAS': 'nro_victimas', 'FECHA': 'fecha', 'HH': 'franja_hora',
               'TIPO_DE_CALLE': 'tipo_calle', 'COMUNA': 'comuna', 'pos x': 'longitud', 'pos y': 'latitud',
               'VICTIMA': 'vehiculo_victima', 'ACUSADO': 'vehiculo_acusado'}

# Convertir a entero

columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

# Categorización de columnas

columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado']



En aras de optimizar y simplificar el conjunto de datos, se ha tomado la decisión de eliminar las columnas 'AAAA', 'MM' y 'DD'. Esta medida se fundamenta en la redundancia de información de fecha presente en la columna 'FECHA'.

La eliminación de estas columnas redundantes conlleva diversos beneficios:

Simplificación del conjunto de datos: Se reduce la complejidad del conjunto, facilitando su comprensión y análisis.

Estandarización de la estructura: Se elimina la inconsistencia en la representación de la fecha, consolidando un único formato en la columna 'FECHA'.

Optimización del espacio: Se reduce el tamaño del conjunto de datos, ahorrando espacio de almacenamiento y mejorando el rendimiento.

Análisis temporal simplificado: La información de fecha centralizada en la columna 'FECHA' facilita la realización de análisis temporales más precisos y eficientes.

In [57]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

homi_hechos_cln = limpiar_data(homi_hechos,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                    
                                    )

In [58]:
homi_hechos_cln.head(20)

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros
5,2016-0008,1,2016-01-24,18,avenida,8,-58.44451316,-34.68475866,moto,objeto fijo
6,2016-0009,1,2016-01-24,19,calle,11,-58.50095869,-34.60825440,moto,auto
7,2016-0010,1,2016-01-29,15,gral paz,9,-58.50877521,-34.66977709,moto,auto
8,2016-0012,1,2016-02-08,1,avenida,1,-58.38048577,-34.61303893,moto,cargas
9,2016-0013,1,2016-02-10,11,avenida,1,-58.39114932,-34.62477387,peaton,auto


In [107]:
rows_with_nulls = homi_hechos_cln[homi_hechos_cln.isnull().any(axis=1)]  # Filtramos las filas con nulos para tomar decisiones

In [108]:
rows_with_nulls

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN
32,2016-0045,1,2016-04-11,10,gral paz,13,-58.46743471,-34.53476874,moto,NaN
35,2016-0049,1,2016-04-17,0,autopista,4,-58.37714647568196,-34.63657525428238,NaN,NaN
38,2016-0052,1,2016-04-20,20,autopista,13,.,.,moto,NaN
57,2016-0080,1,2016-06-18,6,gral paz,9,-58.52927982,-34.63931752,auto,NaN
76,2016-0101,1,2016-08-07,19,gral paz,9,-58.52932872,-34.65014993,NaN,NaN
83,2016-0110,1,2016-08-24,20,avenida,9,-58.51232237,-34.65903530,peaton,NaN
88,2016-0115,1,2016-09-02,11,calle,12,-58.48935041,-34.54917673,NaN,cargas
92,2016-0119,1,2016-09-04,6,avenida,12,-58.49659234,-34.56353635,NaN,NaN
105,2016-0135,1,2016-10-24,5,avenida,14,-58.42260225,-34.56403694,peaton,NaN


Para algunos ids que no tenían latitud y longitud, en particular la comuna "0", se consultó el equivalente a la Autopista 25 de Mayo en https://www.comollegara.com/direccion-a-lat-long.html?lugar=autopista%2025%20de%20mayo%20buenos%20aires. Además se rellenaron valores faltantes para la comuna 13 en cuanto a latitud y longitud

In [59]:
# Reemplazar valores

# Definir los registros que deseas reemplazar
remplazar_reg = [
    {'id_siniestro': '2016-0052', 'comuna': 13, 'longitud': -58.46743471, 'latitud': -34.53476874}, # Comunas 13 
    {'id_siniestro': '2016-0151', 'comuna': 0, 'longitud': -58.369529920, 'latitud': -34.622472560}, # Comunas 0
    {'id_siniestro': '2016-0174', 'comuna': 0, 'longitud': -58.369529920, 'latitud': -34.622472560} # Comunas 0
]

In [60]:
# Iterar sobre los registros y realizar el reemplazo
for registro in remplazar_reg:
    id_siniestro = registro['id_siniestro']
    comuna = registro['comuna']
    longitud = registro['longitud']
    latitud = registro['latitud']

In [61]:
 # Utilizar loc para reemplazar los valores específicos
homi_hechos_cln.loc[homi_hechos_cln['id_siniestro'] == id_siniestro, 'comuna'] = comuna
homi_hechos_cln.loc[homi_hechos_cln['id_siniestro'] == id_siniestro, 'longitud'] = longitud
homi_hechos_cln.loc[homi_hechos_cln['id_siniestro'] == id_siniestro, 'latitud'] = latitud

In [62]:
#Imprimimos el dataframe limpio

homi_hechos_cln.head()

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros


### 3.2. Dataset: Homicidios / victimas

In [63]:
# Acceder al DataFrame por nombre de hoja, Homicidios - Víctimas

homi_victimas = data_homicidios['VICTIMAS']

In [64]:
homi_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [65]:
#Obtenemos información general del df
homi_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [66]:
#anlizamos valores SD para convertir en NaN, repetimos el paso anterior. 

rest_homi_victimas = detectar_sd(homi_victimas)
rest_homi_victimas

,Columna,Cantidad de SD,Porcentaje de SD
5,ROL,11,1.534170
6,VICTIMA,9,1.255230
7,SEXO,6,0.836820
8,EDAD,53,7.391911
9,FECHA_FALLECIMIENTO,68,9.483961


In [67]:
# Reemplazamos 'SD' por NaN en el df

homi_victimas.replace(['SD','sd'], np.nan, inplace=True)

In [68]:
#Preparamos diccionario y listas para la función limpiar_data

# Columnas para pasar a minúcula

columns_to_lower = ['SEXO','ROL']

# Columnas para eliminar

columns_to_drop = ['FECHA','AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO', 'VICTIMA']


# Renombrar columnas

rename_dict = {'ID_hecho': 'id_siniestro', 'SEXO': 'sexo',
               'EDAD': 'edad', 'ROL': 'rol'}

# Convertir a entero

columns_to_int = ['edad']

In [69]:
# Columnas se agrega columna "fatal"

new_columns_dict = {'gravedad': 'fatal'}

In [70]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

homi_victimas_cln = limpiar_data(homi_victimas,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    new_columns=new_columns_dict, # Agregar columnas
                                    )

In [71]:
homi_victimas_cln 

,id_siniestro,rol,sexo,edad,gravedad
0,2016-0001,conductor,masculino,19,fatal
1,2016-0002,conductor,masculino,70,fatal
2,2016-0003,conductor,masculino,30,fatal
3,2016-0004,conductor,masculino,18,fatal
4,2016-0005,conductor,masculino,29,fatal
...,...,...,...,...,...
712,2021-0092,peaton,femenino,50,fatal
713,2021-0093,pasajero_acompañante,femenino,18,fatal
714,2021-0094,pasajero_acompañante,femenino,43,fatal
715,2021-0095,conductor,masculino,27,fatal


In [72]:
# Categorización de columnas

columns_to_categorize = ['sexo','edad','gravedad','rol']
homi_victimas_cln = limpiar_data(homi_victimas_cln, categorize_columns=columns_to_categorize) 

Categorizar las columnas cuando las observaciones no presentan demasiada variación nos ayuda a simplificar y agilizar el procesamiento de los datos.

In [73]:
homi_victimas_cln.info()

print("DataFrame \n limpio ")
homi_victimas_cln.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id_siniestro  717 non-null    object  
 1   rol           706 non-null    category
 2   sexo          711 non-null    category
 3   edad          664 non-null    category
 4   gravedad      717 non-null    category
dtypes: category(4), object(1)
memory usage: 11.7+ KB
DataFrame 
 limpio 


,id_siniestro,rol,sexo,edad,gravedad
0,2016-0001,conductor,masculino,19.0,fatal
1,2016-0002,conductor,masculino,70.0,fatal
2,2016-0003,conductor,masculino,30.0,fatal
3,2016-0004,conductor,masculino,18.0,fatal
4,2016-0005,conductor,masculino,29.0,fatal


### 3.3 Dataset: Lesiones / Hechos

In [74]:
# Acceder al DataFrame por nombre de hoja, Lesiones - Hechos

lesi_hechos = data_lesiones['HECHOS']

In [4]:
lesi_hechos.info()
lesi_hechos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


In [5]:
# Invoca la función 'detectar_sd'

rest_lesi_hechos = detectar_sd(lesi_hechos)
rest_lesi_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,hora,4,0.016817
8,direccion_normalizada,10815,45.469834
9,comuna,846,3.556864
10,tipo_calle,11045,46.436830
11,otra_direccion,18295,76.918226
15,geocodificacion_CABA,1213,5.099853
16,longitud,1209,5.083036
17,latutid,1209,5.083036
18,victima,10733,45.125079
19,acusado,15288,64.275804


In [6]:
# Reemplazar 'SD' por NaN en el df

lesi_hechos.replace(['SD','sd'], np.nan, inplace=True)

Antes de pasar los datos de las listas que contienen las columnas a eliminar y los diccionarios a transformar, haré una valoración de algunos aspectos importantes del df, tales como establecer las cantidades de vehículos involucrados y el número de fallecidos. 

In [13]:
#
lesi_hechos["victima"].value_counts()

victima
MOTO                  5461
CICLISTA              2209
AUTO                  2033
PEATON                2026
TRANSPORTE PUBLICO     678
CAMIONETA              189
TAXI                   182
MOVIL                  100
CAMION                  79
MIXTO                   42
MONOPATIN               28
OTRO                    13
UTILITARIO              12
Name: count, dtype: int64

In [16]:
lesi_hechos["n_victimas"].value_counts() 

n_victimas
1     21032
2      2125
3       400
4       118
5        66
6        21
8        10
7         6
10        4
16        2
0         1
Name: count, dtype: int64

En la celda anterior se evidencia que existe un valor de "0" victimas con un registro, se procede a revisar y se contrasta con el id "PNA-2019-0005246" en el dataset de lesiones y aparece como mujer de 33 años sin datos de lesiones.  

In [17]:
lesi_hechos[lesi_hechos["n_victimas"] == 0] #Se busca en el dataset lesiones mediante el id correspondiente.

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
9928,PNA-2019-0005246,0,2019,12,20,2019-12-20 00:00:00,NaN,NaN,27 DE FEBRERO AV. 6300,NaN,...,NaN,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#Preparamos diccionario y listas para la función limpiar_data

# Columnas para pasar a minúcula

columns_to_lower = ['tipo_calle', 'victima', 'acusado', 'gravedad']

columns_to_drop = ['aaaa','mm', 'dd', 'hora', 'direccion_normalizada', 'otra_direccion', 'calle', 'altura', 'cruce',
                   'participantes', 'geocodificacion_CABA', 'moto', 'auto', 'transporte_publico', 'camion', 'ciclista']

# Reemplazar valores nulos por otros valores
# Según diccionario de datos: Sin datos sobre la gravedad de las lesiones provocadas. 
#A efectos analíticos, los casos sin datos se corresponden con una alta probabilidad a casos leves.

fill_na_dict = {'gravedad': 'leve'}

# Renombrar columnas

rename_dict = {'id': 'id_siniestro', 'n_victimas': 'nro_victimas',
               'latutid': 'latitud', 'victima': 'vehiculo_victima', 'acusado': 'vehiculo_acusado'}

# Reemplazar valores en columnas

values_to_replace = {'comuna': {'No Especificada': np.nan},
                     'nro_victimas': {0: 1}
                    }

# Convertir a flotante

columns_to_float = [ 'latitud', 'longitud']

# Convertir a entero

columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

# Categorización de columnas

columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado', 'gravedad']

In [33]:
# función 'limpiar_data' para la limpieza de los datos 

lesi_hechos_cln = limpiar_data(lesi_hechos,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    fill_na=fill_na_dict, # Rellenar nulos
                                    rename_columns=rename_dict,  # Renombrar columnas                            
                                    replace_values=values_to_replace, # Reemplazar valores                                   
                                    convert_to_float=columns_to_float, # Convertir a float
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

In [34]:

lesi_hechos_cln.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_siniestro      23785 non-null  object  
 1   nro_victimas      23785 non-null  Int64   
 2   fecha             23785 non-null  object  
 3   franja_hora       23776 non-null  Int64   
 4   comuna            22328 non-null  Int64   
 5   tipo_calle        12740 non-null  category
 6   longitud          22314 non-null  float64 
 7   latitud           22314 non-null  float64 
 8   vehiculo_victima  13052 non-null  category
 9   vehiculo_acusado  8497 non-null   category
 10  gravedad          23785 non-null  category
dtypes: Int64(3), category(4), float64(2), object(2)
memory usage: 1.4+ MB


In [35]:
lesi_hechos_cln.head()

,id_siniestro,nro_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latitud,vehiculo_victima,vehiculo_acusado,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,14,NaN,-58.408911,-34.559658,ciclista,NaN,leve
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,8,NaN,-58.443510,-34.669125,auto,NaN,leve
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,8,NaN,-58.468335,-34.677556,NaN,NaN,leve
3,LC-2019-0000079,1,2019-01-01 00:00:00,2,7,NaN,-58.437425,-34.647349,peaton,NaN,leve
4,LC-2019-0000082,4,2019-01-01 00:00:00,4,3,NaN,-58.398225,-34.604579,auto,NaN,leve


In [39]:
#Se corrige la columna fecha que tiene el formato object y se convierte a datetime64[ns]
lesi_hechos_cln["fecha"] = pd.to_datetime(lesi_hechos_cln["fecha"], format="mixed")

In [40]:
lesi_hechos_cln.head()

,id_siniestro,nro_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latitud,vehiculo_victima,vehiculo_acusado,gravedad
0,LC-2019-0000179,1,2019-01-01,9,14,NaN,-58.408911,-34.559658,ciclista,NaN,leve
1,LC-2019-0000053,1,2019-01-01,1,8,NaN,-58.443510,-34.669125,auto,NaN,leve
2,LC-2019-0000063,1,2019-01-01,2,8,NaN,-58.468335,-34.677556,NaN,NaN,leve
3,LC-2019-0000079,1,2019-01-01,2,7,NaN,-58.437425,-34.647349,peaton,NaN,leve
4,LC-2019-0000082,4,2019-01-01,4,3,NaN,-58.398225,-34.604579,auto,NaN,leve


### 3.4. Dataset: Lesiones / Víctimas

In [41]:
# Acceder al DataFrame por nombre de hoja "VICTIMAS"

lesi_victimas = data_lesiones['VICTIMAS']

In [43]:
lesi_victimas.info()
lesi_victimas.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


In [45]:
rest_lesi_victimas = detectar_sd(lesi_victimas)
rest_lesi_victimas

,Columna,Cantidad de SD,Porcentaje de SD
5,VEHICULO_VICTIMA,4479,16.225321
6,SEXO,1929,6.987865
7,EDAD_VICTIMA,3990,14.453903
8,GRAVEDAD,20722,75.066111


In [46]:
# Reemplazar 'SD' por NaN en el df

lesi_victimas.replace(['SD','sd'], np.nan, inplace=True)

In [47]:
#Preparamos diccionario y listas para la función limpiar_data

# Columnas para pasar a minúcula

columns_to_lower = ['SEXO']

# Columnas para eliminar

columns_to_drop = ['FECHA ','AAA', 'MM', 'DD','VEHICULO_VICTIMA','GRAVEDAD']

# Renombrar columnas

rename_dict = {'ID hecho': 'id_siniestro', 'SEXO': 'sexo', 'EDAD_VICTIMA': 'edad'}

# Reemplazar valores en columnas

values_to_replace = {'sexo': {'varon': 'masculino', 'mujer': 'femenino'}}

# Categorización de columnas

columns_to_categorize = ['sexo','edad']

# Columnas para ser agregadas

new_columns_dict = {'rol': ''}

In [48]:
# Invocamos la función "limpiar_data" para limpieza de los datos 

lesi_victimas_cln = limpiar_data(lesi_victimas,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    replace_values=values_to_replace, # Reemplazar valores
                                    categorize_columns=columns_to_categorize, # Categorizar columnas
                                    new_columns=new_columns_dict, # Agregar columnas
                                    )

In [49]:
lesi_victimas_cln

,id_siniestro,sexo,edad,rol
0,LC-2019-0000053,masculino,57,
1,LC-2019-0000063,NaN,NaN,
2,LC-2019-0000079,masculino,NaN,
3,LC-2019-0000082,masculino,45,
4,LC-2019-0000082,femenino,45,
...,...,...,...,...
27600,LC-2021-0451911,masculino,87,
27601,LC-2021-0530228,femenino,60,
27602,LC-2021-0530228,femenino,32,
27603,LC-2021-0201378,masculino,32,


# 4. Unión de datasets

In [75]:
# INNER JOIN en dataframes

homi_hechos_victimas = pd.merge(homi_hechos_cln, homi_victimas_cln, left_on='id_siniestro', right_on='id_siniestro', how='inner')
lesi_hechos_victimas = pd.merge(lesi_hechos_cln, lesi_victimas_cln, left_on='id_siniestro', right_on='id_siniestro', how='inner')

In [76]:
homi_hechos_victimas.head()

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad,gravedad
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto,conductor,masculino,19.0,fatal
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros,conductor,masculino,70.0,fatal
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto,conductor,masculino,30.0,fatal
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN,conductor,masculino,18.0,fatal
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros,conductor,masculino,29.0,fatal


In [77]:
lesi_hechos_victimas.head()

,id_siniestro,nro_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latitud,vehiculo_victima,vehiculo_acusado,gravedad,sexo,edad,rol
0,LC-2019-0000179,1,2019-01-01,9,14,NaN,-58.408911,-34.559658,ciclista,NaN,leve,NaN,NaN,
1,LC-2019-0000053,1,2019-01-01,1,8,NaN,-58.443510,-34.669125,auto,NaN,leve,masculino,57,
2,LC-2019-0000063,1,2019-01-01,2,8,NaN,-58.468335,-34.677556,NaN,NaN,leve,NaN,NaN,
3,LC-2019-0000079,1,2019-01-01,2,7,NaN,-58.437425,-34.647349,peaton,NaN,leve,masculino,NaN,
4,LC-2019-0000082,4,2019-01-01,4,3,NaN,-58.398225,-34.604579,auto,NaN,leve,masculino,45,


In [78]:
#Reorganiza columnas del df "lesi_hechos_victimas"

organizar = ['id_siniestro', 'nro_victimas', 'fecha', 'franja_hora', 'tipo_calle', 'comuna', 'longitud', 'latitud', 
           'vehiculo_victima', 'vehiculo_acusado', 'rol', 'sexo', 'edad', 'gravedad']

lesi_hechos_victimas = lesi_hechos_victimas[organizar]

In [79]:
lesi_hechos_victimas

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad,gravedad
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,-5.840891e+01,-3.455966e+01,ciclista,NaN,,NaN,NaN,leve
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,-5.844351e+01,-3.466913e+01,auto,NaN,,masculino,57,leve
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,-5.846834e+01,-3.467756e+01,NaN,NaN,,NaN,NaN,leve
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,-5.843742e+01,-3.464735e+01,peaton,NaN,,masculino,NaN,leve
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,-5.839822e+01,-3.460458e+01,auto,NaN,,masculino,45,leve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27594,LC-2021-0652865,2,2021-12-31,19,avenida,9,-5.848833e+01,-3.464175e+01,NaN,NaN,,masculino,28,leve
27595,LC-2021-0652865,2,2021-12-31,19,avenida,9,-5.848833e+01,-3.464175e+01,NaN,NaN,,masculino,52,leve
27596,LC-2021-0652907,1,2021-12-31,20,NaN,1,-5.838289e+01,-3.458308e+01,NaN,NaN,,masculino,23,leve
27597,LC-2021-0652921,1,2021-12-31,22,calle,5,-5.841453e+01,-3.461429e+01,moto,transporte publico,,masculino,31,grave


# 6. Exportar datasets

In [80]:
# Almacenar DataFrames en un nuevo archivo CSV

#homi_hechos_victimas.to_csv('DataSets/homicidios_cln.csv', index=False)
#lesi_hechos_victimas.to_csv('DataSets/lesiones_cln.csv', index=False)